In [ ]:
import numpy as np
import PyPhenom as ppi
from datetime import datetime
from Auto_SEM.autonomous_sem.phenom import SEM

In [ ]:
# parameter settings
row = 10
col = 10
image_save_dir = ''

In [ ]:
def imaging(sem_instance: SEM, cur_row: int, cur_col: int, image_save_dir: str):
    acqScanParams = ppi.ScanParams()
    acqScanParams.size = ppi.Size(1920, 1200)
    acqScanParams.detector = ppi.DetectorMode.All
    acqScanParams.nFrames = 16
    acqScanParams.hdr = False
    acqScanParams.scale = 1.0
    acq = sem_instance.phenom.SemAcquireImage(acqScanParams)
    acq.metadata.displayWidth = 0.5
    acq.metadata.dataBarLabel = "Label"
    acqWithDatabar = ppi.AddDatabar(acq)
    file_name = f'{datetime.now().strftime("%Y%m%d_%H%M%S")}_row-{cur_row}_col-{cur_col}.tiff'
    file_path = f'{image_save_dir}/{file_name}'
    ppi.Save(acqWithDatabar, file_path)

In [ ]:
sem = SEM()
horizontal_field_width = sem.phenom.GetHFW()
vertical_field_width = horizontal_field_width * 9 / 16
X_abs, Y_abs = sem.phenom.GetStageModeAndPosition().position

# create a matrix list of points with row x col, with interval of horizontal and vertical field width, starting at the current position
X = np.linspace(X_abs, X_abs + horizontal_field_width * (col - 1), col)
Y = np.linspace(Y_abs, Y_abs - vertical_field_width * (row - 1), row)

# iterate through the grid and take a picture at each point, starting from the top left to the right, then down
for i in range(row):
    for j in range(col):
        sem.phenom.MoveTo(X[j], Y[i])
        sem.phenom.SemAutoFocus()
        sem.phenom.SemAutoContrastBrightness()
        imaging(
            sem_instance=sem,
            cur_row=i,
            cur_col=j,
            image_save_dir=image_save_dir
        )
        print(f'Image taken at row {i}, col {j}')
